In [1]:
from Data import Data
import numpy as np
import pandas as pd
from numpy import arctan2, sin, cos

In [2]:
df = pd.read_pickle(".\sample.pkl")
df.head()

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   

         avg_speed  
id time             
1  0.00  35.366264  
   0.04  35.366264  
   0.08  35.366264  
   0.12  35.366264  
   0.16  35.366264

In [3]:
# https://towardsdatascience.com/calculating-the-bearing-between-two-geospatial-coordinates-66203f57e4b4

def bearing(c1, c2):
    lat1,lon1 = c1
    lat2,lon2 = c2
    
    dL = lon2 - lon1
    x = cos(lon2) * sin(dL)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dL)
    return arctan2(x,y)

In [4]:
def bearing_df(df,idx):
    
    df_1 = df.loc[idx]
    index_list = df.index[df.index.isin([idx],level = 0)]
    df_1 = df_1.set_index(pd.Index(range(0,len(df_1.index))))
    df_2 = df_1.set_index(df_1.index-1)
    df_2 = df_2.drop(-1)
    dL = df_2["lon"] - df_1["lon"]

    x = cos(df_2["lon"]) * sin(dL)
    y = cos(df_1["lat"]) * sin(df_2["lat"]) - sin(df_1["lat"]) * cos(df_2["lat"]) * cos(dL)
    df_bearing = arctan2(x,y)
    df_bearing.index = index_list
    return df_bearing



In [5]:
DT = 0.04
EPS = 1e-3

def coord_at_idx(df, idx):
    lat = float(df.loc[[idx], 'lat'])
    lon = float(df.loc[[idx], 'lon'])
    return (lat,lon)

def bearing_at_idx(df, idx):
    id,time = idx
    next_idx = (id, time+DT)

    if not df.index.isin([next_idx]).any():
        return None
    if float(df.loc[[idx], 'speed']) < EPS:
        return None

    a = coord_at_idx(df, idx)
    b = coord_at_idx(df, next_idx)
    return bearing(a,b)

bearing_at_idx(df, (2,0.08))

-0.33168503526379606

In [7]:
df2 = df.copy()
df3 = df.copy()

#df2['bearing'] = None
df3['bearing'] = None
# this is too slow
import time
start_time = time.time()


for idx in df2.index:
   df2.loc[[idx], 'bearing'] = bearing_at_idx(df2,idx)
    
print("timetaken to run old code %s " % (time.time() - start_time))
start_time = time.time()

bearing_list = [bearing_df(df3,idx) for idx in df3.index.unique(level=0)]
df3['bearing'] = pd.concat(bearing_list)
    
print("timetaken to run new code %s " % (time.time() - start_time))   


timetaken to run old code 49.77890181541443 
timetaken to run new code 0.032958984375 


In [154]:
df3["bearing"]-df2["bearing"]

id  time 
1   0.00     0.0
    0.04     0.0
    0.08     0.0
    0.12     0.0
    0.16     0.0
            ... 
5   29.24    NaN
    29.28    0.0
    29.32    0.0
    29.36    0.0
    29.40    NaN
Name: bearing, Length: 3460, dtype: object